# Pruebas entrenamiento multisalida

In [31]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn.utils import shuffle
import pandas as pd
import numpy as np

Se importa el csv

In [32]:
df = pd.read_csv('gameStates.csv', sep=',',header=None)

In [33]:
df.columns = ['timeStamp','Px', 'Py','heat','Pup1x' ,'Pup1y','Pup2x','Pup2y', 'E1x', 'E1y', 'E1h', 'E2x', 'E2y','E2h','E3x', 'E3y','E3h','E4x', 'E4y','E4h','E5x', 'E5y','E5h','E6x', 'E6y','E6h',"VKey","HKey","Shooting"]
train_data = df.drop(["Shooting", "VKey", "HKey"], axis =1)
target_data =df[["VKey","HKey","Shooting"]].copy()

In [34]:
train_data[:5]

,timeStamp,Px,Py,heat,Pup1x,Pup1y,Pup2x,Pup2y,E1x,E1y,...,E3h,E4x,E4y,E4h,E5x,E5y,E5h,E6x,E6y,E6h
0,86171327,0.0,2.18,0.0,999.0,999.0,999.0,999.0,1.81,9.46,...,0,999.0,999.0,0,999.0,999.0,0,999,999,0
1,86171327,0.0,2.18,0.0,999.0,999.0,999.0,999.0,1.79,9.32,...,0,999.0,999.0,0,999.0,999.0,0,999,999,0
2,86171327,0.0,2.18,0.0,999.0,999.0,999.0,999.0,1.79,9.17,...,0,999.0,999.0,0,999.0,999.0,0,999,999,0
3,86171328,0.0,2.18,0.0,999.0,999.0,999.0,999.0,1.79,9.03,...,0,999.0,999.0,0,999.0,999.0,0,999,999,0
4,86171328,0.0,2.18,0.0,999.0,999.0,999.0,999.0,1.81,8.89,...,0,999.0,999.0,0,999.0,999.0,0,999,999,0


In [35]:
forest = RandomForestClassifier(n_estimators=100, random_state=1)

In [36]:
X=train_data.values.astype(float)
Y=target_data.values.astype(str)

In [37]:
def transformaEje(value):
    if value == "UpArrow" or value == "RightArrow":
        return 1
    elif value == "DownArrow" or value == "LeftArrow":
        return -1
    else:
        return 0 

def transformaDisparo(value):
    if value:
        return 1
    else:
        return 0

In [38]:
#target_data["VKey"] =
target_data["VKey"] = target_data["VKey"].map(transformaEje)
target_data["HKey"] = target_data["HKey"].map(transformaEje)
target_data["Shooting"] = target_data["Shooting"].map(transformaDisparo)
#target_data["VKey"].map(transformaEje)[:10]
target_data[:20]

,VKey,HKey,Shooting
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,-1,1,0
6,-1,1,1
7,0,-1,1
8,0,1,1
9,0,1,0


In [39]:
forest = forest.fit(X, target_data)


In [40]:
#print(forest.predict(X[:50]))
Y = target_data.values
Y

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ..., 
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [41]:
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(forest, X, Y, cv=5)


In [42]:
from sklearn import metrics

print(metrics.accuracy_score(Y[:,0], predicted[:,0]))
print(metrics.accuracy_score(Y[:,1], predicted[:,1]))
print(metrics.accuracy_score(Y[:,2], predicted[:,2]))

0.625706940874
0.480719794344
0.598114824336


In [1]:
import pandas as pd
import numpy as np
import itertools


In [21]:
df = pd.read_csv('gameStates.csv', sep=',',header=None)
df.columns = ['timeStamp', 'Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh2', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'E3h', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6',"VKey","HKey","Shooting"]
#Preprocesado sin valorar vida restante
hist = df[['Ex1', 'Ey1', 'Ex2', 'Ey2', 'Ex3', 'Ey3','Ex4', 'Ey4','Ex5', 'Ey5']][:20:2].copy()

def histo2DRow(row):
   
    x = row.filter(regex=('Ex'))
    y = row.filter(regex=('Ey'))
    
    xedges = [-3,-2,  -1, 0,   1,  2,3]
    yedges = [ 0, 2.5, 5, 7.5, 10]

    H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
    H = H.T  # Let each row list bins with common y range.
    
    return pd.Series(H.reshape(-1))

hist = hist.apply(histo2DRow, axis = 1)
hist



,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
oldNames = list(range(24))
newNames = [str(x)+"/"+str(y) for x,y in itertools.product(np.arange(-3,3), np.arange(0,10,2.5))]
hist.rename(columns=dict(zip(oldNames,newNames)))


,-3/0.0,-3/2.5,-3/5.0,-3/7.5,-2/0.0,-2/2.5,-2/5.0,-2/7.5,-1/0.0,-1/2.5,...,0/5.0,0/7.5,1/0.0,1/2.5,1/5.0,1/7.5,2/0.0,2/2.5,2/5.0,2/7.5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
